In [ ]:
# default_exp models.XResNet1dPlus

# XResNet1dPlus

> This is a modified version of fastai's XResNet model in github. Changes include:
* API is modified to match the default timeseriesAI's API.
* (Optional) Uber's CoordConv 1d

In [ ]:
#export
from tsai.imports import *
from tsai.models.layers import *
from tsai.models.utils import *

In [ ]:
#export
class XResNet1dPlus(nn.Sequential):
    @delegates(ResBlock1dPlus)
    def __init__(self, block, expansion, layers, fc_dropout=0.0, c_in=3, n_out=1000, stem_szs=(32,32,64),
                 widen=1.0, sa=False, act_cls=defaults.activation, ks=3, stride=2, coord=False, **kwargs):
        store_attr('block,expansion,act_cls,ks')
        if ks % 2 == 0: raise Exception('kernel size has to be odd!')
        stem_szs = [c_in, *stem_szs]
        stem = [ConvBlock(stem_szs[i], stem_szs[i+1], ks=ks, coord=coord, stride=stride if i==0 else 1,
                          act=act_cls)
                for i in range(3)]

        block_szs = [int(o*widen) for o in [64,128,256,512] +[256]*(len(layers)-4)]
        block_szs = [64//expansion] + block_szs
        blocks    = self._make_blocks(layers, block_szs, sa, coord, stride, **kwargs)
        backbone = nn.Sequential(*stem, MaxPool(ks=ks, stride=stride, padding=ks//2, ndim=1), *blocks)
        head = nn.Sequential(AdaptiveAvgPool(sz=1, ndim=1), Flatten(), nn.Dropout(fc_dropout),
                             nn.Linear(block_szs[-1]*expansion, n_out))
        super().__init__(OrderedDict([('backbone', backbone), ('head', head)]))
        self._init_cnn(self)

    def _make_blocks(self, layers, block_szs, sa, coord, stride, **kwargs):
        return [self._make_layer(ni=block_szs[i], nf=block_szs[i+1], blocks=l, coord=coord, 
                                 stride=1 if i==0 else stride, sa=sa and i==len(layers)-4, **kwargs)
                for i,l in enumerate(layers)]

    def _make_layer(self, ni, nf, blocks, coord, stride, sa, **kwargs):
        return nn.Sequential(
            *[self.block(self.expansion, ni if i==0 else nf, nf, coord=coord, stride=stride if i==0 else 1,
                      sa=sa and i==(blocks-1), act_cls=self.act_cls, ks=self.ks, **kwargs)
              for i in range(blocks)])
    
    def _init_cnn(self, m):
        if getattr(self, 'bias', None) is not None: nn.init.constant_(self.bias, 0)
        if isinstance(self, (nn.Conv1d,nn.Conv2d,nn.Conv3d,nn.Linear)): nn.init.kaiming_normal_(self.weight)
        for l in m.children(): self._init_cnn(l)

In [ ]:
#export
def _xresnetplus(expansion, layers, **kwargs):
    return XResNet1dPlus(ResBlock1dPlus, expansion, layers, **kwargs)

In [ ]:
#export
@delegates(ResBlock)
def xresnet1d18plus (c_in, c_out, act=nn.ReLU, **kwargs): return _xresnetplus(1, [2, 2,  2, 2], c_in=c_in, n_out=c_out, act_cls=act, **kwargs)
@delegates(ResBlock)
def xresnet1d34plus (c_in, c_out, act=nn.ReLU, **kwargs): return _xresnetplus(1, [3, 4,  6, 3], c_in=c_in, n_out=c_out, act_cls=act, **kwargs)
@delegates(ResBlock)
def xresnet1d50plus (c_in, c_out, act=nn.ReLU, **kwargs): return _xresnetplus(4, [3, 4,  6, 3], c_in=c_in, n_out=c_out, act_cls=act, **kwargs)
@delegates(ResBlock)
def xresnet1d101plus (c_in, c_out, act=nn.ReLU, **kwargs): return _xresnetplus(4, [3, 4, 23, 3], c_in=c_in, n_out=c_out, act_cls=act, **kwargs)
@delegates(ResBlock)
def xresnet1d152plus (c_in, c_out, act=nn.ReLU, **kwargs): return _xresnetplus(4, [3, 8, 36, 3], c_in=c_in, n_out=c_out, act_cls=act, **kwargs)
@delegates(ResBlock)
def xresnet1d18_deepplus (c_in, c_out, act=nn.ReLU, **kwargs): return _xresnetplus(1, [2,2,2,2,1,1], c_in=c_in, n_out=c_out, act_cls=act, **kwargs)
@delegates(ResBlock)
def xresnet1d34_deepplus (c_in, c_out, act=nn.ReLU, **kwargs): return _xresnetplus(1, [3,4,6,3,1,1], c_in=c_in, n_out=c_out, act_cls=act, **kwargs)
@delegates(ResBlock)
def xresnet1d50_deepplus (c_in, c_out, act=nn.ReLU, **kwargs): return _xresnetplus(4, [3,4,6,3,1,1], c_in=c_in, n_out=c_out, act_cls=act, **kwargs)
@delegates(ResBlock)
def xresnet1d18_deeperplus (c_in, c_out, act=nn.ReLU, **kwargs): return _xresnetplus(1, [2,2,1,1,1,1,1,1], c_in=c_in, n_out=c_out, act_cls=act, **kwargs)
@delegates(ResBlock)
def xresnet1d34_deeperplus (c_in, c_out, act=nn.ReLU, **kwargs): return _xresnetplus(1, [3,4,6,3,1,1,1,1], c_in=c_in, n_out=c_out, act_cls=act, **kwargs)
@delegates(ResBlock)
def xresnet1d50_deeperplus (c_in, c_out, act=nn.ReLU, **kwargs): return _xresnetplus(4, [3,4,6,3,1,1,1,1], c_in=c_in, n_out=c_out, act_cls=act, **kwargs)

In [ ]:
net = xresnet1d18plus(3, 2, coord=True)
x = torch.rand(32, 3, 50)
net(x)

tensor([[-0.0198,  0.0322],
        [ 0.0740,  0.1516],
        [ 0.0818,  0.0663],
        [ 0.2270, -0.1658],
        [-0.0347,  0.0498],
        [-0.5321,  0.0969],
        [-0.0886,  0.1850],
        [ 0.1142,  0.1375],
        [-0.4652,  0.7560],
        [-0.0749,  0.3875],
        [-0.1290, -0.2009],
        [-0.2658, -0.0194],
        [-0.4039,  0.0762],
        [ 0.0060,  0.5701],
        [-0.1219,  0.6144],
        [-0.0630,  0.2920],
        [-0.2875,  0.3220],
        [ 0.0418,  0.2288],
        [ 0.0716, -0.0458],
        [-0.2295, -0.0607],
        [-0.0861,  0.0701],
        [-0.1200,  0.3971],
        [-0.2329, -0.1175],
        [-0.1352, -0.2607],
        [-0.1228,  0.3307],
        [-0.2759,  0.0553],
        [-0.0906,  0.5389],
        [-0.0633, -0.0666],
        [ 0.0336,  0.5431],
        [ 0.0036, -0.1895],
        [ 0.3577,  0.2622],
        [-0.2278,  0.3647]], grad_fn=<AddmmBackward>)

In [ ]:
bs, c_in, seq_len = 2, 4, 32
c_out = 2
x = torch.rand(bs, c_in, seq_len)
archs = [
    xresnet1d18plus, xresnet1d34plus, xresnet1d50plus, 
    xresnet1d18_deepplus, xresnet1d34_deepplus, xresnet1d50_deepplus, xresnet1d18_deeperplus,
    xresnet1d34_deeperplus, xresnet1d50_deeperplus
#     # Long test
#     xresnet1d101, xresnet1d152,
]
for i, arch in enumerate(archs):
    print(i, arch.__name__)
    test_eq(arch(c_in, c_out, sa=True, act=Mish, coord=True)(x).shape, (bs, c_out))

0 xresnet1d18plus
1 xresnet1d34plus
2 xresnet1d50plus
3 xresnet1d18_deepplus
4 xresnet1d34_deepplus
5 xresnet1d50_deepplus
6 xresnet1d18_deeperplus
7 xresnet1d34_deeperplus
8 xresnet1d50_deeperplus


In [ ]:
m = xresnet1d34plus(4, 2, act=Mish)
test_eq(len(get_layers(m, is_bn)), 38)
test_eq(check_weight(m, is_bn)[0].sum(), 22)

In [ ]:
# hide
out = create_scripts()
beep(out)

<IPython.core.display.Javascript object>

Converted 000_utils.ipynb.
Converted 000b_data.validation.ipynb.
Converted 000c_data.preparation.ipynb.
Converted 001_data.external.ipynb.
Converted 002_data.core.ipynb.
Converted 003_data.preprocessing.ipynb.
Converted 003b_data.transforms.ipynb.
Converted 003c_data.mixed_augmentation.ipynb.
Converted 003d_data.image.ipynb.
Converted 003e_data.features.ipynb.
Converted 005_data.tabular.ipynb.
Converted 006_data.mixed.ipynb.
Converted 007_metrics.ipynb.
Converted 008_learner.ipynb.
Converted 009_optimizer.ipynb.
Converted 010_callback.core.ipynb.
Converted 011_callback.noisy_student.ipynb.
Converted 012_callback.gblend.ipynb.
Converted 013_callback.TSBERT.ipynb.
Converted 100_models.layers.ipynb.
Converted 100b_models.utils.ipynb.
Converted 100c_models.explainability.ipynb.
Converted 101_models.ResNet.ipynb.
Converted 101b_models.ResNetPlus.ipynb.
Converted 102_models.InceptionTime.ipynb.
Converted 102b_models.InceptionTimePlus.ipynb.
Converted 103_models.MLP.ipynb.
Converted 103b_mode